from google.colab import drive
drive.mount('/content/drive')

%pip install openai
%pip install ffmpeg-python

In [4]:
import os
from utils.extractframes import FrameExtractor
import graphene
from gpt_ask import run_gpt
directory_path = 'eval/quant'


In [5]:
import cv2
import time
def run_oic():
    output_dir = "out"
    
    cap = cv2.VideoCapture()
    cap.opne("https://interactions.ics.unisg.ch/61-102/cam1/live-stream")
    
    
    
    for video in os.scandir(directory_path):
            if video.is_file():
                if not video.name.startswith('.'):
                    ex = FrameExtractor(directory_path+'/'+video.name, 10, 5)
                    ex.main()
                    
                    # instantiate graphene "OIC core that runs RelTR"  
                    g = graphene.Graphene(alpha=0.3, min_assignment_conf=0.6)
                    
                    # check if the out directory exists else create one
                    if not os.path.isdir(output_dir):
                        os.mkdir(output_dir)
                      
                    # prepare output files 
                    text = os.path.splitext(video.name)[0]+'graph2text.txt'
                    img_dir_path = directory_path+'/'+os.path.splitext(video.name)[0]+'-opencv'
        
                      
                    # classify images from the image directory
                    #g.classify_images(image_path=img_dir_path) # without windowing
                    start_1 = time.time()
                    g.classify_images_window(image_path=img_dir_path, window_size=5)
                    print("Time for classification "+time.strftime("%H:%M:%S", time.gmtime(time.time()-start_1)))
                    # generate relationship graph
                    graph_dir_path = img_dir_path + "/img/JSON"
                    
                    start_2 = time.time()
                    g.generate_temporal_graph_frames_no_plot(graph_dir_path, img_dir_path + "/img")
                    print("Time for generating frame graph"+time.strftime("%H:%M:%S", time.gmtime(time.time()-start_2)))
                    # save textual output in the out directory
                    g.tg.to_text(os.path.join(output_dir, text))
          
                    if os.path.isfile(os.path.join(output_dir,text)):
                        with open(os.path.join(output_dir,text)) as f:
                            prompt_content = "".join(map(str, f.readlines()))
                            return str(prompt_content)
                    else:
                        continue

In [6]:
if os.path.isdir(directory_path):
  prompt = run_oic()
  if prompt is not None:
    choice_string = "1. laptop, 2. refrigerator, 3. Door, 4. Bag"
    start_3 = time.time()
    answer = run_gpt(prompt, 'What object was opened by the Person? Guess the most likely answer among these four options: '+choice_string+' Respond only with a single number between 1 and 4. Do not produce any other output.')
    print("Time for llm qa "+time.strftime("%H:%M:%S", time.gmtime(time.time()-start_3)))
    print('OIC answer: {}'.format(answer))
  else:
    print('OIC answer: {}'.format(None))
      

100%|██████████| 70/70 [00:00<00:00, 6063.76it/s]


copying images and cleaning up temporary files:
eval/quant/Interaction_T1_78OY3LS-opencv/frame0-00-00.18.jpg eval/quant/Interaction_T1_78OY3LS-opencv/img/frame0-00-00.18.jpg
eval/quant/Interaction_T1_78OY3LS-opencv/frame0-00-00.73.jpg eval/quant/Interaction_T1_78OY3LS-opencv/img/frame0-00-00.73.jpg
eval/quant/Interaction_T1_78OY3LS-opencv/frame0-00-01.18.jpg eval/quant/Interaction_T1_78OY3LS-opencv/img/frame0-00-01.18.jpg
eval/quant/Interaction_T1_78OY3LS-opencv/frame0-00-01.73.jpg eval/quant/Interaction_T1_78OY3LS-opencv/img/frame0-00-01.73.jpg
eval/quant/Interaction_T1_78OY3LS-opencv/frame0-00-02.18.jpg eval/quant/Interaction_T1_78OY3LS-opencv/img/frame0-00-02.18.jpg
eval/quant/Interaction_T1_78OY3LS-opencv/frame0-00-02.73.jpg eval/quant/Interaction_T1_78OY3LS-opencv/img/frame0-00-02.73.jpg
eval/quant/Interaction_T1_78OY3LS-opencv/frame0-00-03.18.jpg eval/quant/Interaction_T1_78OY3LS-opencv/img/frame0-00-03.18.jpg
eval/quant/Interaction_T1_78OY3LS-opencv/frame0-00-03.73.jpg eval/quan

In [7]:
import shutil

shutil.rmtree('eval/quant/car_out-opencv/')


FileNotFoundError: [Errno 2] No such file or directory: 'eval/quant/car_out-opencv/'

In [ ]:
import shutil

shutil.rmtree('temp')


In [8]:
import os
import time
from gpt_ask import run_gpt
with open(os.path.join('/Users/sanjivjha/Developer/IMP-OIC/out/Interaction_T1_78OY3LSgraph2text.txt')) as f:
      context = "".join(map(str,f.readlines()))

      choice_string = "1. laptop, 2. refrigerator, 3. Door, 4. Bag"
      start_3 = time.time()
      answer = run_gpt(context, 'What object was opened by the Person? Guess the most likely answer among these four options: '+choice_string+' Respond only with a single number between 1 and 4. Do not produce any other output.')
      print("time for for llm qa "+time.strftime("%H:%M:%S", time.gmtime(time.time()-start_3)))
      if answer is not None:
        print('OIC answer: {}'.format(answer))
      else:
        print('OIC answer: {}'.format(None))



time for for llm qa 00:00:00
OIC answer: 3


In [3]:
import spacy

# Load the English language model in spaCy
nlp = spacy.load("en_core_web_sm")

# Given words
words = ["This", "is", "an", "example", "sentence"]

# Join the words to form a sentence
sentence = ' '.join(words)

# Parse the sentence using spaCy
parsed_sentence = nlp(sentence)

# Get the text representation of the parsed sentence
processed_sentence = parsed_sentence.text

print(processed_sentence)

  ERROR: HTTP error 404 while getting https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.2/en_core_web_sm-3.7.2.tar.gz
ERROR: Could not install requirement https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.2/en_core_web_sm-3.7.2.tar.gz because of HTTP error 404 Client Error: Not Found for url: https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.2/en_core_web_sm-3.7.2.tar.gz for URL https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.2/en_core_web_sm-3.7.2.tar.gz
Note: you may need to restart the kernel to use updated packages.


OSError: [E053] Could not read config file from /Users/sanjivjha/anaconda3/lib/python3.8/site-packages/en_core_web_sm/en_core_web_sm-2.2.0/config.cfg